# Peak Signal-to-Noise Ratio (PSNR) 

#### Source: https://ffmpeg.org/ffmpeg-all.html#psnr

Peak signal-to-noise ratio (PSNR) is the ratio between the maximum possible power of a signal and the power of corrupting noise that affects the fidelity of its representation. Because many signals have a very wide dynamic range, PSNR is usually expressed as a logarithmic quantity using the decibel scale [1](https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio).

 - [1] http://en.wikipedia.org/wiki/Hausdorff_distance 

## Example:

This ffmpeg filter takes in input two input videos, the first input is the distorted and the second input is used as a "reference" video for computing the PSNR.
As the distorted video has a different resolution, the first step is to upscale:

In [ ]:
!ffmpeg -hide_banner -y -r 25 -i videos/dist/x264/bbb_640x360_CRF40.mp4 -vf scale=1920x1080 -c:v libx264 -crf 0 /tmp/dist.mp4

Also it assumes that both inputs have the same number of frames, which are compared one by one (Ffmpeg, 2021). Then the PSNR can be computed with the following command:

In [ ]:
!mkdir -p results/psnr && \
ffmpeg -hide_banner -r 25 -i /tmp/dist.mp4 -r 25 -i videos/ref/bbb_1920x1080.mp4 -lavfi \
    "[0:v]settb=AVTB,setpts=PTS-STARTPTS[distorted];\
     [1:v]settb=AVTB,setpts=PTS-STARTPTS[reference];\
     [distorted][reference]psnr=f=results/psnr/bbb_psnr.txt" -f null -

## Graphical representation.

In [ ]:
import pandas as pd 
import re
import plotly.express as px

# File with the PSNR results (adapt to your needs)
file='results/psnr/bbb_psnr.txt'

path_file = open(file, 'r')
lines = path_file.readlines()

psnr_frames=[]

for line in lines:
    float_frame=re.sub("[ :]", ",", line)
    lts_frame=float_frame.split(',')
    psnr_frames.append([float(lts_frame[11]),float(lts_frame[13]),
                        float(lts_frame[15]),float(lts_frame[17])])
    
df=pd.DataFrame(psnr_frames, columns=['PSNR','Y','U','V'])

fig = px.line(df,x=df.index, y='PSNR',
              title='PSNR by frame',
              labels={"index": "Frame"}, 
              template='simple_white',
              markers=True)

fig.add_hline(y=df.PSNR.mean(), line_width=2, line_dash='dash')

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True)

fig.show()
   

This second plot shows the PSNR and the PSN per component (Y, U, V)

In [ ]:
import pandas as pd 
import re
import plotly.graph_objects as go

# File with the PSNR results (adapt to your needs)
file='results/psnr/bbb_psnr.txt'

path_file = open(file, 'r')
lines = path_file.readlines()

psnr_frames=[]

for line in lines:
    float_frame=re.sub("[ :]", ",", line)
    lts_frame=float_frame.split(',')
    psnr_frames.append([float(lts_frame[11]),float(lts_frame[13]),
                        float(lts_frame[15]),float(lts_frame[17])])
    
df=pd.DataFrame(psnr_frames, columns=['PSNR','Y','U','V'])

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df.PSNR,
                    mode='lines+markers',
                    name='PSNR',
                         line=dict(color='#3E4680')))
fig.add_trace(go.Scatter(x=df.index, y=df.Y,
                    mode='lines+markers',
                    name='Y',
                         line=dict(color='#FFD0A1')))
fig.add_trace(go.Scatter(x=df.index, y=df.U,
                    mode='lines+markers',
                    name='U',
                          line=dict(color='#C56E90')))
fig.add_trace(go.Scatter(x=df.index, y=df.V,
                    mode='lines+markers',
                    name='V',
                          line=dict(color='#80BFA0')))

fig.update_layout(
    title={'text':'<b>PSNR by frame</b>',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True,
    template='simple_white',
    xaxis_title='Frame',yaxis_title='PSNR')

fig.show()